In [1]:
### GROUP 2 ###

# B.J.T. Donkers - 1506897
# S. van der Leek - 1564226
# L. Sloot - 1365711
# S. Diemel  - 1645749
# A.E. Murariu - 1532081
# Z.V. Petrov - 1554999

In [2]:
# imports
import pandas as pd
from datetime import datetime
import re

In [3]:
# read the data (2012)
df = pd.read_csv('data/BPI_Challenge_2012-training.csv')

df_test = pd.read_csv('data/BPI_Challenge_2012-test.csv')

# read the data (road traffic fines)
# df = pd.read_csv('data/Road_Traffic_Fine_Management_Process-training.csv')

# read the data (2017)
# df = pd.read_csv('data/BPI Challenge 2017-training.csv')

# read the data (2018)
# df = pd.read_csv('data/BPI Challenge 2018-training.csv')

df

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256
...,...,...,...,...,...,...,...
214372,38835094290529,201854,2012-01-18T02:09:07.029+01:00,50000,O_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214373,38835094290528,201854,2012-01-18T02:09:07.029+01:00,50000,A_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214374,38835094290530,201854,2012-01-18T02:09:07.029+01:00,50000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 15:30:23.187
214375,35858681954366,199678,2012-01-10T19:16:52.800+01:00,30000,W_Nabellen offertes,START,14-03-2012 15:36:15.299


In [4]:
# Defining database-specific variables
case_column = "case concept:name"
registration_time_column = "case REG_DATE"
event_column = "event concept:name"
timestamp_column = "event time:timestamp"
amount_column = 'case AMOUNT_REQ'
row_nr_column = 'row_nr'
index_column = "index"
timeformat_registration = "%Y-%m-%dT%H:%M:%S" # new time format
timeformat_timestamp = "%d-%m-%Y %H:%M:%S.%f"
lifecycle_column = 'event lifecycle:transition' # additional info for the timePrediction_onTheFly, set to None if you don't want to use it
amount_column_in_data = True

# Names of new columns we will add in the notebooks
position_column = "Position" # added in this tool notebook
baseline_la_next_event_column = "Baseline Last Event Prediction for Next Activity" # added in the baseline notebook
baseline_la_next_timestamp_column = "Baseline Last Event Prediction for Next Timestamp" # added in the baseline notebook
baseline_pos_next_event_column = "Baseline Last Pos Prediction for Next Activity" # added in the baseline notebook
baseline_pos_next_timestamp_column = "Baseline Last Pos Prediction for Next Timestamp" # added in the baseline notebook
pred_event_otf = "OTF Prediction for Next Activity" # added by event_onthefly notebook
otf_next_timestamp_column = "OnTheFly Prediction for Next Timestamp" # added by the timePrediction_onTheFly notebook

In [5]:
# Basic data preprocessing of the timestamps
df[registration_time_column]= [re.sub('\..*|\+.*','',a,flags=re.DOTALL) for a in df[registration_time_column]]
df[registration_time_column] = [datetime.strptime(date, timeformat_registration) for date in df[registration_time_column]]
df[timestamp_column] = [datetime.strptime(date, timeformat_timestamp) for date in df[timestamp_column]]
df = df.sort_values(by=[case_column, timestamp_column]).reset_index() # sort values by user and time of event

# Basic data preprocessing of the timestamps
df_test[registration_time_column]= [re.sub('\..*|\+.*','',a,flags=re.DOTALL) for a in df_test[registration_time_column]]
df_test[registration_time_column] = [datetime.strptime(date, timeformat_registration) for date in df_test[registration_time_column]]
df_test[timestamp_column] = [datetime.strptime(date, timeformat_timestamp) for date in df_test[timestamp_column]]
df_test = df_test.sort_values(by=[case_column, timestamp_column]).reset_index() # sort values by user and time of event

In [6]:
# We fill in the Position column that shows which position is a certain event in the trace (the first event is 1)
df[position_column] = df.groupby([case_column]).cumcount()+1
df_test[position_column] = df_test.groupby([case_column]).cumcount()+1

In [7]:
# Running the notebook that adds the baseline predictions to df and df_test
%run -i "Baseline Prediction Tool.ipynb"

In [8]:
# Accuracy % of the baseline based on last event, event predictions:

training_event_accuracy = len(df[(df[baseline_la_next_event_column].shift(periods=1)==df[event_column]) & (df[case_column].shift(periods=1) == df[case_column])]) * 100 / len(df[df[case_column].shift(periods=1) == df[case_column]])

test_event_accuracy = len(df_test[(df_test[baseline_la_next_event_column].shift(periods=1)==df_test[event_column]) & (df_test[case_column].shift(periods=1) == df_test[case_column])]) * 100 / len(df_test[df_test[case_column].shift(periods=1) == df_test[case_column]])

training_event_accuracy, test_event_accuracy

(64.9130980638327, 64.06370976661874)

In [9]:
# Accuracy % of the baseline based on last position, event predictions:

training_event_accuracy = len(df[(df[baseline_pos_next_event_column].shift(periods=1)==df[event_column]) & (df[case_column].shift(periods=1) == df[case_column])]) * 100 / len(df[df[case_column].shift(periods=1) == df[case_column]])

test_event_accuracy = len(df_test[(df_test[baseline_pos_next_event_column].shift(periods=1)==df_test[event_column]) & (df_test[case_column].shift(periods=1) == df_test[case_column])]) * 100 / len(df_test[df_test[case_column].shift(periods=1) == df_test[case_column]])

training_event_accuracy, test_event_accuracy

(45.370461188379075, 45.16756995907532)

In [10]:
# Mean Absolute Error of the baseline based on last event, time predictions:

training_time_MAE = abs(df[timestamp_column] - df[baseline_la_next_timestamp_column].shift(periods=1))[df[case_column].shift(periods=1) == df[case_column]].mean()

test_time_MAE = abs(df_test[timestamp_column] - df_test[baseline_la_next_timestamp_column].shift(periods=1))[df_test[case_column].shift(periods=1) == df_test[case_column]].mean()

training_time_MAE, test_time_MAE

(Timedelta('0 days 16:20:59.739888261'),
 Timedelta('0 days 14:43:13.416911984'))

In [11]:
# Mean Absolute Error of the baseline based on last position, time predictions:

training_time_MAE = abs(df[timestamp_column] - df[baseline_pos_next_timestamp_column].shift(periods=1))[df[case_column].shift(periods=1) == df[case_column]].mean()

test_time_MAE = abs(df_test[timestamp_column] - df_test[baseline_pos_next_timestamp_column].shift(periods=1))[df_test[case_column].shift(periods=1) == df_test[case_column]].mean()

training_time_MAE, test_time_MAE

(Timedelta('0 days 17:35:28.176916022'),
 Timedelta('0 days 16:08:20.277217351'))

In [12]:
# Running the notebook that adds the on-the-fly event prediction to df and df_test
%run -i "event_onTheFly.ipynb"

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', 'sys', '_i', '_ii', '_iii', '_i1', '_i2', 'pd', 'datetime', 're', 'site', 'os', '__vsc_ipynb_file__', '_i3', 'df', 'df_test', '_3', '_i4', 'case_column', 'registration_time_column', 'event_column', 'timestamp_column', 'amount_column', 'row_nr_column', 'index_column', 'timeformat_registration', 'timeformat_timestamp', 'additionalInfo', 'lifecycle_column', 'amount_column_in_data', 'position_column', 'baseline_la_next_event_column', 'baseline_la_next_timestamp_column', 'baseline_pos_next_event_column', 'baseline_pos_next_timestamp_column', 'pred_event_otf', 'otf_next_timestamp_column', '_i5', '_i6', '_i7', 'shifted_deltatimes', 'replacenat', 'unique_events', 'dict_common_next_event', 'dict_time_per_event', 'dict_events_after_pos', 'count_events_after_pos', 'dict_time_per_pos', 'training_event_accuracy',

C:\Users\Sander\AppData\Local\Temp/ipykernel_5132/1514717183.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[row_nr_column] = dataset.index
C:\Users\Sander\AppData\Local\Temp/ipykernel_5132/1514717183.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_test[row_nr_column] = dataset_test.index
C:\Users\Sander\AppData\Local\Temp/ipykernel_5132/1514717183.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

   case concept:name      event concept:name event lifecycle:transition  \
0             173688             A_SUBMITTED                   COMPLETE   
1             173688       A_PARTLYSUBMITTED                   COMPLETE   
2             173688           A_PREACCEPTED                   COMPLETE   
3             173688  W_Completeren aanvraag                   SCHEDULE   
4             173688  W_Completeren aanvraag                      START   

   case AMOUNT_REQ  row_nr  
0            20000       0  
1            20000       1  
2            20000       2  
3            20000       3  
4            20000       4  
dict: Elapsed time during the whole program in seconds: 82.5625
train accuracy:0.8083792690821351
length df = 214377, length predictions = 214377
test accuracy:0.814378940382701
length df = 47823, length predictions = 47823


In [13]:
df.head(10)

,index,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,Position,Baseline Last Event Prediction for Next Activity,Baseline Last Event Prediction for Next Timestamp,Baseline Last Pos Prediction for Next Activity,Baseline Last Pos Prediction for Next Timestamp
0,0,0,173688,2011-10-01 00:38:44,20000,A_SUBMITTED,COMPLETE,2011-10-01 00:38:44.546,1,A_PARTLYSUBMITTED,2011-10-01 00:38:45.128640462,A_PARTLYSUBMITTED,2011-10-01 00:38:45.128640462
1,1,1,173688,2011-10-01 00:38:44,20000,A_PARTLYSUBMITTED,COMPLETE,2011-10-01 00:38:44.880,2,W_Afhandelen leads,2011-10-01 00:39:19.982779539,W_Afhandelen leads,2011-10-01 00:39:19.982779539
2,2,2,173688,2011-10-01 00:38:44,20000,A_PREACCEPTED,COMPLETE,2011-10-01 00:39:37.906,3,W_Completeren aanvraag,2011-10-01 00:39:38.408771753,W_Afhandelen leads,2011-10-01 03:30:53.567069032
3,3,3,173688,2011-10-01 00:38:44,20000,W_Completeren aanvraag,SCHEDULE,2011-10-01 00:39:38.875,4,W_Completeren aanvraag,2011-10-01 17:58:08.032379243,W_Completeren aanvraag,2011-10-01 03:55:31.785088057
4,89,4,173688,2011-10-01 00:38:44,20000,W_Completeren aanvraag,START,2011-10-01 11:36:46.437,5,W_Completeren aanvraag,2011-10-02 04:55:15.594379243,W_Completeren aanvraag,2011-10-01 11:54:36.483873248
5,94,5,173688,2011-10-01 00:38:44,20000,A_ACCEPTED,COMPLETE,2011-10-01 11:42:43.308,6,O_SELECTED,2011-10-01 11:46:27.513278604,W_Completeren aanvraag,2011-10-01 15:21:10.772730102
6,98,7,173688,2011-10-01 00:38:44,20000,A_FINALIZED,COMPLETE,2011-10-01 11:45:09.243,7,O_CREATED,2011-10-01 11:45:11.557131461,W_Completeren aanvraag,2011-10-01 12:49:36.989151176
7,99,6,173688,2011-10-01 00:38:44,20000,O_SELECTED,COMPLETE,2011-10-01 11:45:09.243,8,O_CREATED,2011-10-01 11:45:11.199693633,W_Completeren aanvraag,2011-10-01 15:50:33.483132306
8,100,8,173688,2011-10-01 00:38:44,20000,O_CREATED,COMPLETE,2011-10-01 11:45:11.197,9,O_SENT,2011-10-01 11:45:11.257123109,W_Completeren aanvraag,2011-10-01 14:35:59.469570225
9,101,9,173688,2011-10-01 00:38:44,20000,O_SENT,COMPLETE,2011-10-01 11:45:11.380,10,W_Nabellen offertes,2011-10-01 11:45:11.587961660,W_Completeren aanvraag,2011-10-01 15:40:21.236745516


In [14]:
print(f'train accuracy event on the fly: {train_acc_event_otf*100}') # does still run because is in event_onthefly.ipynb
print(f'test accuracy event on the fly: {test_acc_event_otf*100}')

train accuracy event on the fly: 80.8379269082135
test accuracy event on the fly: 81.4378940382701


In [15]:
#df = df.sort_values(by=[case_column, timestamp_column]).reset_index(drop=True)
#df_test = df_test.sort_values(by=[case_column, timestamp_column]).reset_index(drop=True)

#training_event_accuracy = len(df[(df[pred_event_otf].shift(periods=1)==df[event_column]) & (df[case_column].shift(periods=1) == df[case_column])]) * 100 / len(df[df[case_column].shift(periods=1) == df[case_column]])

#test_event_accuracy = len(df_test[(df_test[pred_event_otf].shift(periods=1)==df_test[event_column]) & (df_test[case_column].shift(periods=1) == df_test[case_column])]) * 100 / len(df_test[df_test[case_column].shift(periods=1) == df_test[case_column]])

#training_event_accuracy, test_event_accuracy


In [16]:
# Accuracy % of the on-the-fly event predictions:

# training_event_accuracy = len(df[(df[baseline_next_event_column].shift(periods=1)==df[event_column]) & (df[case_column].shift(periods=1) == df[case_column])]) * 100 / len(df[df[case_column].shift(periods=1) == df[case_column]])

# test_event_accuracy = len(df_test[(df_test[baseline_next_event_column].shift(periods=1)==df_test[event_column]) & (df_test[case_column].shift(periods=1) == df_test[case_column])]) * 100 / len(df_test[df_test[case_column].shift(periods=1) == df_test[case_column]])

# training_event_accuracy, test_event_accuracy

In [17]:
# Running the notebook that adds the on-the-fly event prediction to df and df_test
%run -i "timePrediction_onTheFly.ipynb" # Takes around 2 minutes to run

All columns are available in the dataframe


NameError: name 'df_validation' is not defined

NameError: name 'df_validation' is not defined

In [ ]:
df.head(10)

In [ ]:
# Mean Absolute Error of the on-the-fly time predictions:

# training_time_MAE = abs(df[timestamp_column] - df[baseline_next_timestamp_column].shift(periods=1))[df[case_column].shift(periods=1) == df[case_column]].mean()

# test_time_MAE = abs(df_test[timestamp_column] - df_test[baseline_next_timestamp_column].shift(periods=1))[df_test[case_column].shift(periods=1) == df_test[case_column]].mean()

# training_time_MAE, test_time_MAE